In [ ]:
imgfolder = '/mnt/d/autocls11/imgs/cmb18/i_20180818/'
thold = [0.02, 0.05, 0.1, 0.2, 0.3]
output   = '/mnt/d/autocls11/satseg/seg_20180818.tif'

In [ ]:
import os
import datetime as dt

from glass.wenv.grs import run_grass
from glass.pys.oss import lst_ff, fprop
from glass.pys.tm import now_as_str

In [ ]:
imgs = lst_ff(imgfolder, file_format='tif')
imgs = [imgs[0]]

In [ ]:
imgs

In [ ]:
ws = os.path.dirname(output)
if not os.path.exists(ws):
    mkdir(ws, overwrite=None)
    
"""
Start GRASS GIS Session
"""

loc = f'loc_{now_as_str()}'
grsb = run_grass(
    ws, grassBIN='grass78', location=loc,
    srs=imgs[0]
)

import grass.script.setup as gsetup
    
gsetup.init(grsb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.rst.seg import img_segment

In [ ]:
rsts = [rst_to_grs(i) for i in imgs]

In [ ]:
bname = fprop(output, 'fn')
segs = []
for t in range(len(thold)):
    a = dt.datetime.now().replace(microsecond=0)
    seg = img_segment(
        rsts, thold[t], 1.5,
        f'{bname}_{str(t).replace(".", "")}',
        method=None,
        seeds=None if not t else segs[t-1],
        ascmd=True
    )
    
    grs_to_rst(
        seg, os.path.join(ws, f"{seg}.tif"),
        as_cmd=True, is_int=True
    )
    
    segs.append(seg)
    b = dt.datetime.now().replace(microsecond=0)
    
    print(b-a)